## Import Statements

In [194]:
import os
import json
import shutil
from typing import Dict, List, Any, Optional, Tuple
from datetime import datetime

# Core LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor
from langgraph.checkpoint.memory import MemorySaver

# OpenAI and other LLM imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# Research and crawling tools
import tavily
from tavily import TavilyClient
import asyncio
import nest_asyncio
from crawl4ai import AsyncWebCrawler
from crawl4ai.async_configs import BrowserConfig, CrawlerRunConfig

# For image processing and markdown
import re
import requests
from pathlib import Path
import markdown
from bs4 import BeautifulSoup

# load env
from dotenv import load_dotenv
load_dotenv()

True

In [195]:
nest_asyncio.apply()

In [196]:
tavily_api_key = os.getenv("TAVILY_API_KEY")
tavily_client = TavilyClient(api_key=tavily_api_key)

In [197]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [198]:
# Define the state structure
class BlogWritingState(BaseModel):
    topic: str = Field(description="The blog topic to write about")
    search_results: List[Dict[str, Any]] = Field(default_factory=list, description="Results from Tavily search")
    scraped_content: Dict[str, Dict[str, Any]] = Field(default_factory=dict, description="Content scraped from URLs")  # Changed type here
    research_summary: str = Field(default="", description="Summary of research findings")
    outline: List[Dict[str, Any]] = Field(default_factory=list, description="Outline of the blog post")
    draft: str = Field(default="", description="Draft of the blog post in markdown")
    critique: str = Field(default="", description="Critique of the blog post")
    final_draft: str = Field(default="", description="Final version of the blog post")
    evaluation: Dict[str, Any] = Field(default_factory=dict, description="Evaluation metrics and feedback")
    images: List[Dict[str, Any]] = Field(default_factory=list, description="Images to include in the blog")
    output_dir: str = Field(default="", description="Directory to save the blog post and assets")
    error: Optional[str] = Field(default=None, description="Error message if any")

In [199]:
class ResearchTool:
    """Tool for researching a topic using Tavily."""
    
    def __init__(self, client):
        self.client = client
    
    def search(self, topic: str, max_results: int = 5) -> List[Dict]:
        """Search for relevant articles on the topic."""
        print(f"Searching for articles on: {topic}")
        try:
            search_result = self.client.search(query=f"latest articles and research on {topic}", 
                                              search_depth="advanced",
                                              max_results=max_results)
            return search_result.get("results", [])
        except Exception as e:
            return [{"error": f"Search failed: {str(e)}"}]

research_tool = ResearchTool(tavily_client)

In [200]:
class WebCrawlerTool:
    """Tool for crawling and extracting content from websites."""
    
    async def crawl_url(self, url: str) -> Dict:
        """Crawl a URL and extract its content."""
        print(f"Crawling URL: {url}")
        try:
            browser_config = BrowserConfig()
            run_config = CrawlerRunConfig()
            
            async with AsyncWebCrawler(config=browser_config) as crawler:
                result = await crawler.arun(url=url, config=run_config)
                
                # Extract images if available
                images = []
                soup = BeautifulSoup(markdown.markdown(result.markdown), 'html.parser')
                for img in soup.find_all('img'):
                    if img.get('src'):
                        images.append({
                            "url": img.get('src'),
                            "alt": img.get('alt', ''),
                            "source_url": url
                        })
                
                return {
                    "content": result.markdown,
                    "images": images,
                    "url": url,
                    "title": result.metadata.get("title", "")
                }
        except Exception as e:
            return {"error": f"Failed to crawl {url}: {str(e)}", "url": url}

    def crawl_urls(self, urls: List[str]) -> Dict[str, Any]:
        """Crawl multiple URLs and aggregate their content."""
        print(f"Crawling {len(urls)} URLs")
        results = {}
        
        async def process_urls():
            tasks = [self.crawl_url(url) for url in urls]
            return await asyncio.gather(*tasks)
        
        crawl_results = asyncio.run(process_urls())
        
        for result in crawl_results:
            url = result.get("url")
            if url:
                results[url] = result
        
        return results

web_crawler_tool = WebCrawlerTool()

In [201]:
class ImageDownloader:
    """Tool for downloading and saving images."""
    
    
    def download_image(self, image_url: str, output_path: str) -> str:
        """Download an image and save it to the specified path."""
        print(f"Downloading image from: {image_url}")
        try:
            response = requests.get(image_url, stream=True)
            if response.status_code == 200:
                with open(output_path, 'wb') as f:
                    shutil.copyfileobj(response.raw, f)
                return output_path
            else:
                return ""
        except Exception:
            return ""

image_downloader = ImageDownloader()

In [202]:
# Agent definitions
class ResearchAgent:
    """Agent responsible for collecting research on a topic."""
    
    def __init__(self, llm):
        self.llm = llm
        self.research_tool = research_tool
        self.web_crawler_tool = web_crawler_tool
    
    def research_prompt(self):
        return ChatPromptTemplate.from_messages([
            ("system", """You are a professional research agent. Your task is to gather relevant information on a given topic 
            for writing a high-quality blog post. Search for the most relevant and up-to-date information. 
            Focus on finding authoritative sources, recent developments, key concepts, and insightful perspectives."""),
            ("human", "Research the topic: {topic}"),
            MessagesPlaceholder(variable_name="history")
        ])
    
    def summarize_prompt(self):
        return ChatPromptTemplate.from_messages([
            ("system", """You are a research summarizer. Given content collected from multiple sources, create a comprehensive
            and well-organized summary that captures the key information, trends, varied perspectives, and important details
            on the topic. Format your summary in clear sections with markdown. Include specific facts, statistics, and quotes
            that would be valuable for writing an authoritative blog post."""),
            ("human", "Summarize the following research on {topic}:\n\n{research_content}"),
        ])
    
    def run(self, state: BlogWritingState) -> Dict:
        """Run the research process and update the state."""
        print("Running Research Agent")
        try:
            topic = state.topic  # Use dot notation instead of dict access
            
            # Search for relevant articles
            search_results = self.research_tool.search(topic)
            
            # Extract URLs from search results
            urls = [result.get("url") for result in search_results if result.get("url")]
            
            # Crawl the URLs to get content
            scraped_content = self.web_crawler_tool.crawl_urls(urls)
            
            # Extract all images from scraped content
            all_images = []
            for url, data in scraped_content.items():
                if "images" in data and isinstance(data["images"], list):
                    for img in data["images"]:
                        img["source_url"] = url
                        all_images.append(img)
            
            # Create a research content string from scraped content
            research_content = "\n\n===SOURCE===\n\n".join([
                f"URL: {url}\nTITLE: {data.get('title', 'No Title')}\n\n{data.get('content', 'No content')}"
                for url, data in scraped_content.items() if "error" not in data
            ])
            
            # Summarize the research
            summary_chain = self.summarize_prompt() | self.llm | StrOutputParser()
            research_summary = summary_chain.invoke({"topic": topic, "research_content": research_content})
            
            # Return updated state (as a dict that will be converted to new state)
            return {
                "topic": topic,
                "search_results": search_results,
                "scraped_content": scraped_content,
                "research_summary": research_summary,
                "images": all_images
            }
        except Exception as e:
            return {"error": f"Research agent error: {str(e)}"}

In [203]:
class OutlineAgent:
    """Agent responsible for creating a blog post outline."""
    
    def __init__(self, llm):
        self.llm = llm
    
    def outline_prompt(self):
        return ChatPromptTemplate.from_messages([
            ("system", """You are an expert blog outline creator. Given a topic and research summary, create a comprehensive
            outline for a professional blog post. The outline should include:
            1. An engaging title
            2. A compelling introduction section
            3. Main sections with clear headings
            4. Subsections where appropriate
            5. A conclusion section
            6. Call-to-action or next steps
            
            Format your outline as JSON with the following structure:
            {
                "title": "Main Blog Title",
                "sections": [
                    {
                        "heading": "Section Heading",
                        "subheadings": ["Subheading 1", "Subheading 2"],
                        "key_points": ["Point 1", "Point 2"]
                    }
                ]
            }"""),
            ("human", """Create an outline for a professional blog post on the topic: {topic}
            
            Here is the research summary to work with:
            
            {research_summary}"""),
        ])
    
    def run(self, state: Dict) -> Dict:
        """Create an outline based on the research and update the state."""
        print("Running Outline Agent")
        try:
            outline_chain = self.outline_prompt() | self.llm | JsonOutputParser()
            outline = outline_chain.invoke({
                "topic": state.topic,
                "research_summary": state.research_summary
            })
            
            return {**state, "outline": outline}
        except Exception as e:
            return {**state, "error": f"Outline agent error: {str(e)}"}

In [204]:
class WriterAgent:
    """Agent responsible for writing the blog post."""
    
    def __init__(self, llm):
        self.llm = llm
    
    def writing_prompt(self):
        return ChatPromptTemplate.from_messages([
            ("system", """You are a professional blog writer with expertise in creating engaging, informative, and well-structured
            content. Write a comprehensive blog post following the provided outline and incorporating the research summary.
            
            Guidelines:
            - Write in a clear, engaging, and authoritative tone
            - Use markdown formatting for headings, emphasis, lists, etc.
            - Include code snippets where relevant (with proper markdown formatting)
            - Suggest places to insert images with image descriptions in this format: ![Description](image_placeholder)
            - Back claims with data from the research
            - Make the introduction compelling to hook readers
            - End with a strong conclusion and call-to-action
            - Target a professional audience with technical accuracy but accessible explanation
            - Aim for ~1500-2000 words of comprehensive coverage"""),
            ("human", """Write a professional blog post on the topic: {topic}
            
            Outline:
            {outline}
            
            Research Summary:
            {research_summary}
            
            Available images (suggest where to use these in your blog post):
            {images_text}"""),
        ])
    
    def run(self, state: Dict) -> Dict:
        """Write the blog post draft and update the state."""
        print("Running Writer Agent")
        try:
            # Format images info for the prompt
            images_text = "\n".join([
                f"- Image {i+1}: {img.get('alt', 'No description')} (from {img.get('source_url', 'unknown')})"
                for i, img in enumerate(state["images"][:5])  # Limit to 5 images to avoid token limits
            ])
            
            outline_str = json.dumps(state.outline, indent=2) if isinstance(state.outline, dict) else str(state.outline)
            
            writing_chain = self.writing_prompt() | self.llm | StrOutputParser()
            draft = writing_chain.invoke({
                "topic": state.topic,
                "outline": outline_str,
                "research_summary": state.research_summary,
                "images_text": images_text or "No images available."
            })
            
            return {**state, "draft": draft}
        except Exception as e:
            return {**state, "error": f"Writer agent error: {str(e)}"}

In [205]:
class CritiqueAgent:
    """Agent responsible for critiquing the blog post."""
    
    def __init__(self, llm):
        self.llm = llm
    
    def critique_prompt(self):
        return ChatPromptTemplate.from_messages([
            ("system", """You are an expert content editor and critic. Your job is to provide constructive criticism and 
            improvement suggestions for blog posts. Evaluate the blog draft on:
            
            1. Content depth and accuracy
            2. Structure and flow
            3. Engagement and reader appeal
            4. Technical correctness
            5. Language and clarity
            6. SEO optimization
            7. Use of evidence and examples
            
            Be specific about what works well and what could be improved. Provide actionable recommendations."""),
            ("human", """Review this blog draft on the topic "{topic}":
            
            {draft}
            
            Provide a comprehensive critique with specific recommendations for improvement."""),
        ])
    
    def run(self, state: Dict) -> Dict:
        """Critique the blog draft and update the state."""
        print("Running Critique Agent")
        try:
            critique_chain = self.critique_prompt() | self.llm | StrOutputParser()
            critique = critique_chain.invoke({
                "topic": state.topic,
                "draft": state.draft
            })
            
            return {**state, "critique": critique}
        except Exception as e:
            return {**state, "error": f"Critique agent error: {str(e)}"}

In [206]:
class RevisionAgent:
    """Agent responsible for revising the blog post based on critique."""
    
    def __init__(self, llm):
        self.llm = llm
    
    def revision_prompt(self):
        return ChatPromptTemplate.from_messages([
            ("system", """You are an expert content reviser. Your task is to improve a blog draft based on editorial critique.
            Make substantive improvements to:
            
            1. Address all critique points thoroughly
            2. Enhance the structure and flow
            3. Improve language, clarity, and engagement
            4. Strengthen evidence and examples
            5. Optimize for SEO and readability
            
            Maintain the markdown formatting and structure of the original while improving the content.
            Keep image placeholders with improved descriptions where appropriate."""),
            ("human", """Revise this blog draft on "{topic}" based on the following critique:
            
            --- CRITIQUE ---
            {critique}
            
            --- ORIGINAL DRAFT ---
            {draft}
            
            Provide a comprehensive revision that addresses all critique points."""),
        ])
    def run(self, state: Dict) -> Dict:
        """Revise the blog draft based on critique and update the state."""
        print("Running Revision Agent")
        try:
            revision_chain = self.revision_prompt() | self.llm | StrOutputParser()
            final_draft = revision_chain.invoke({
                "topic": state.topic,
                "draft": state.draft,
                "critique": state.crtique
            })
            
            return {**state, "final_draft": final_draft}
        except Exception as e:
            return {**state, "error": f"Revision agent error: {str(e)}"}

In [207]:
class EvaluationAgent:
    """Agent responsible for evaluating the final blog post."""
    
    def __init__(self, llm):
        self.llm = llm
    
    def evaluation_prompt(self):
        return ChatPromptTemplate.from_messages([
            ("system", """You are an expert content evaluator. Your task is to provide a comprehensive evaluation of a blog post
            based on the following criteria:
            
            1. Content Quality (1-10): Depth, accuracy, and value to the reader
            2. Structure (1-10): Organization, flow, and logical progression
            3. Engagement (1-10): Hook, storytelling, and reader interest
            4. Technical Accuracy (1-10): Correctness of information and concepts
            5. Language & Style (1-10): Clarity, tone, and professionalism
            6. SEO Potential (1-10): Keyword usage, meta elements, and searchability
            7. Overall Score (1-10): Holistic assessment
            
            For each criterion, provide a numerical score and brief justification.
            Then provide 2-3 strongest aspects and 2-3 areas for improvement.
            
            Format your response as JSON with the following structure:
            {
                "scores": {
                    "content_quality": {"score": X, "justification": "..."},
                    "structure": {"score": X, "justification": "..."},
                    "engagement": {"score": X, "justification": "..."},
                    "technical_accuracy": {"score": X, "justification": "..."},
                    "language_style": {"score": X, "justification": "..."},
                    "seo_potential": {"score": X, "justification": "..."},
                    "overall": {"score": X, "justification": "..."}
                },
                "strengths": ["Strength 1", "Strength 2", "Strength 3"],
                "improvements": ["Improvement 1", "Improvement 2", "Improvement 3"]
            }"""),
            ("human", """Evaluate this final blog post on the topic "{topic}":
            
            {final_draft}"""),
        ])
    
    def run(self, state: Dict) -> Dict:
        """Evaluate the final blog post and update the state."""
        print("Running Evaluation Agent")
        try:
            evaluation_chain = self.evaluation_prompt() | self.llm | JsonOutputParser()
            evaluation = evaluation_chain.invoke({
                "topic": state.topic,
                "final_draft": state.final_draft
            })
            
            return {**state, "evaluation": evaluation}
        except Exception as e:
            return {**state, "error": f"Evaluation agent error: {str(e)}"}

In [208]:
class ExportAgent:
    """Agent responsible for exporting the blog post and assets."""
    
    def __init__(self):
        self.image_downloader = ImageDownloader()
    
    def create_output_directory(self, topic: str) -> str:
        """Create an output directory for the blog post and its assets."""
        # Create a sanitized directory name from the topic
        dir_name = re.sub(r'[^\w\s-]', '', topic).strip().lower()
        dir_name = re.sub(r'[-\s]+', '-', dir_name)
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        output_dir = f"blog_export_{dir_name}_{timestamp}"
        
        # Create directories
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
        
        return output_dir
    
    def process_markdown_images(self, markdown_text: str, images: List[Dict], output_dir: str) -> str:
        """Process image placeholders in markdown and download images."""
        img_dir = os.path.join(output_dir, "images")
        
        # Find image placeholders in markdown
        img_placeholders = re.findall(r'!\[(.*?)\]\((image_placeholder)\)', markdown_text)
        
        for i, (alt_text, _) in enumerate(img_placeholders):
            if i < len(images):
                img = images[i]
                img_url = img.get("url")
                if img_url:
                    # Generate a filename
                    img_filename = f"image_{i+1}.jpg"
                    img_path = os.path.join(img_dir, img_filename)
                    
                    # Download the image
                    result = self.image_downloader.download_image(img_url, img_path)
                    
                    if result:
                        # Replace placeholder with relative path
                        markdown_text = markdown_text.replace(
                            f"![{alt_text}](image_placeholder)",
                            f"![{alt_text}](images/{img_filename})"
                        )
        
        return markdown_text
    
    def create_evaluation_report(self, evaluation: Dict, output_dir: str) -> None:
        """Create an evaluation report markdown file."""
        if not evaluation:
            return
        
        report_path = os.path.join(output_dir, "evaluation_report.md")
        
        try:
            with open(report_path, "w", encoding="utf-8") as f:
                f.write("# Blog Evaluation Report\n\n")
                
                # Write scores
                f.write("## Scores\n\n")
                scores = evaluation.get("scores", {})
                for criterion, data in scores.items():
                    score = data.get("score", "N/A")
                    justification = data.get("justification", "No justification provided")
                    f.write(f"### {criterion.replace('_', ' ').title()}\n")
                    f.write(f"**Score:** {score}/10\n\n")
                    f.write(f"**Justification:** {justification}\n\n")
                
                # Write strengths
                f.write("## Strengths\n\n")
                for strength in evaluation.get("strengths", ["No strengths identified"]):
                    f.write(f"- {strength}\n")
                
                # Write improvements
                f.write("\n## Areas for Improvement\n\n")
                for improvement in evaluation.get("improvements", ["No improvements identified"]):
                    f.write(f"- {improvement}\n")
        except Exception as e:
            print(f"Error creating evaluation report: {str(e)}")
    
    def run(self, state: Dict) -> Dict:
        """Export the blog post and assets to local folder."""
        print("Running Export Agent")
        try:
            # Create output directory
            output_dir = self.create_output_directory(state["topic"])
            
            # Process markdown and download images
            final_markdown = self.process_markdown_images(
                state["final_draft"], 
                state["images"], 
                output_dir
            )
            
            # Write the final blog post
            blog_path = os.path.join(output_dir, "blog_post.md")
            with open(blog_path, "w", encoding="utf-8") as f:
                f.write(final_markdown)
            
            # Create evaluation report
            self.create_evaluation_report(state["evaluation"], output_dir)
            
            # Write metadata
            metadata_path = os.path.join(output_dir, "metadata.json")
            metadata = {
                "topic": state["topic"],
                "created_at": datetime.now().isoformat(),
                "title": state["outline"].get("title", state["topic"]) if isinstance(state["outline"], dict) else state["topic"],
                "evaluation_summary": {
                    "overall_score": state["evaluation"].get("scores", {}).get("overall", {}).get("score", "N/A") 
                    if isinstance(state["evaluation"], dict) else "N/A"
                }
            }
            
            with open(metadata_path, "w", encoding="utf-8") as f:
                json.dump(metadata, f, indent=2)
            
            return {**state, "output_dir": output_dir}
        except Exception as e:
            return {**state, "error": f"Export agent error: {str(e)}"}

In [ ]:
def setup_blog_writing_graph():
    """Set up the LangGraph for the blog writing process."""
    # Initialize agents
    research_agent = ResearchAgent(llm)
    outline_agent = OutlineAgent(llm)
    writer_agent = WriterAgent(llm)
    critique_agent = CritiqueAgent(llm)
    revision_agent = RevisionAgent(llm)
    evaluation_agent = EvaluationAgent(llm)
    export_agent = ExportAgent()
    
    # Define the state graph
    workflow = StateGraph(BlogWritingState)

    # Add nodes with different names than the state keys
    workflow.add_node("research_node", lambda state: research_agent.run(state))
    workflow.add_node("outline_node", lambda state: outline_agent.run(state))
    workflow.add_node("draft_node", lambda state: writer_agent.run(state))
    workflow.add_node("critique_node", lambda state: critique_agent.run(state))
    workflow.add_node("revision_node", lambda state: revision_agent.run(state))
    workflow.add_node("evaluation_node", lambda state: evaluation_agent.run(state))
    workflow.add_node("export_node", lambda state: export_agent.run(state))

    # Define edges (workflow)
    workflow.set_entry_point("research_node")
    workflow.add_edge("research_node", "outline_node")
    workflow.add_edge("outline_node", "draft_node")
    workflow.add_edge("draft_node", "critique_node")
    workflow.add_edge("critique_node", "revision_node")
    workflow.add_edge("revision_node", "evaluation_node")
    workflow.add_edge("evaluation_node", "export_node")
    workflow.add_edge("export_node", END)
    
    # Error handling (optional)
    def check_for_errors(state):
        if state.error:
            return "error_handler"
        else:
            return "continue_flow"  # Named return value instead of None

    workflow.add_conditional_edges(
        "research_node",
        check_for_errors,
        {
            "error_handler": END,
            "continue_flow": "outline_node"  # Use the named value instead of None
        }
    )

    
    # Compile the graph
    return workflow.compile()

In [210]:
class BlogWritingApp:
    """Main application class for the blog writing system."""
    
    def __init__(self):
        self.graph = setup_blog_writing_graph()
        # Set up checkpointing to save the state
        self.memory = MemorySaver()
    
    def write_blog(self, topic: str) -> Dict:
        """Run the blog writing process for a given topic."""
        print(f"Starting blog writing process for topic: {topic}")
        
        # Initialize state
        initial_state = {"topic": topic}
        
        # Run the graph
        try:
            # Stream events and handle them appropriately
            for output in self.graph.stream(initial_state, {"configurable": {"checkpointer": self.memory}}):
                # Check what type of object output is
                if isinstance(output, dict):
                    # If it's a dict and contains 'event', handle LangGraph events
                    if 'event' in output:
                        event_type = output.get('event')
                        if event_type == "on_chain_start":
                            print(f"Starting: {output.get('name', 'unknown')}")
                        elif event_type == "on_chain_end":
                            print(f"Completed: {output.get('name', 'unknown')}")
                        elif event_type == "on_chain_error":
                            print(f"Error in {output.get('name', 'unknown')}: {output.get('error', 'unknown error')}")
                    else:
                        # If it's a dict without 'event', it might be state data
                        print("Processing step completed")
                else:
                    # Not a dict
                    print(f"Processing: {str(output)[:50]}...")
                    
            # Get the final state
            final_state = self.memory.get_latest()
            
            if final_state.error:
                print(f"Process completed with error: {final_state.error}")
            else:
                print(f"Blog writing process completed successfully!")
                print(f"Output directory: {final_state.output_dir}")
            
            # Convert to dict for easier consumption
            return dict(final_state)
            
        except Exception as e:
            print(f"Error during blog writing process: {str(e)}")
            return {"error": str(e)}

In [211]:
# Input and output guardrails
class BlogInputValidator:
    """Validator for blog writing inputs."""
    
    def validate_topic(self, topic: str) -> Tuple[bool, str]:
        """Validate the blog topic."""
        if not topic or len(topic.strip()) < 3:
            return False, "Topic must be at least 3 characters long."
        
        if len(topic) > 200:
            return False, "Topic is too long. Please limit to 200 characters."
        
        return True, ""


In [212]:
# Demo usage
def run_demo():
    """Run a demonstration of the blog writing system."""
    # Set up the application
    app = BlogWritingApp()
    validator = BlogInputValidator()
    
    # Get user input
    topic = input("Enter blog topic: ")
    
    # Validate input
    is_valid, error_message = validator.validate_topic(topic)
    if not is_valid:
        print(f"Invalid input: {error_message}")
        return
    
    # Run the blog writing process
    result = app.write_blog(topic)
    
    print("\nProcess completed!")
    
    # Print summary - safer access with defaults
    if isinstance(result, dict):
        # Check if output directory was created
        output_dir = result.get("output_dir")
        if output_dir:
            print(f"Blog post saved to: {output_dir}/blog_post.md")
            print(f"Evaluation report: {output_dir}/evaluation_report.md")
            
            # Show evaluation summary if available
            evaluation = result.get("evaluation", {})
            if isinstance(evaluation, dict) and "scores" in evaluation:
                scores = evaluation.get("scores", {})
                if "overall" in scores:
                    print(f"\nOverall score: {scores['overall'].get('score', 'N/A')}/10")
                
                strengths = evaluation.get("strengths", [])
                if strengths:
                    print("\nStrengths:")
                    for strength in strengths[:3]:
                        print(f"- {strength}")
        else:
            print("No output directory was created. Check for errors in the process.")
            error = result.get("error", "Unknown error")
            if error:
                print(f"Error: {error}")
    else:
        print("Unexpected result format. Process may not have completed successfully.")

if __name__ == "__main__":
    run_demo()

Starting blog writing process for topic: Model Context Protocol
Running Research Agent
Searching for articles on: Model Context Protocol
Crawling 5 URLs
Crawling URL: https://wandb.ai/onlineinference/mcp/reports/The-Model-Context-Protocol-MCP-by-Anthropic-Origins-functionality-and-impact--VmlldzoxMTY5NDI4MQ
Crawling URL: https://www.anthropic.com/news/model-context-protocol
Crawling URL: https://medium.com/@subashpalvel/the-hidden-blueprint-of-ai-how-the-model-context-protocol-shapes-our-digital-future-95d4af401dff
Crawling URL: https://medium.com/@alekseyrubtsov/the-revolutionary-impact-of-model-context-protocol-mcp-on-working-with-llms-5a85d4330185
Crawling URL: https://nshipster.com/model-context-protocol/
[INIT].... → Crawl4AI 0.5.0.post4
[INIT].... → Crawl4AI 0.5.0.post4
[INIT].... → Crawl4AI 0.5.0.post4
[INIT].... → Crawl4AI 0.5.0.post4
[INIT].... → Crawl4AI 0.5.0.post4
[FETCH]... ↓ https://medium.com/@alekseyrubtsov/the-revolutiona... | Status: True | Time: 1.44s
[SCRAPE].. ◆ ht